In [1]:
from interpretml_utils import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Loading dataset
### (Taiwan)

In [4]:
data = pd.read_csv("../data/UCI_Credit_Card.csv")
data = data.drop(columns=["ID"])
data = data.rename(columns={"default.payment.next.month": "default"})

X = data.drop(columns=["default"])
y = data["default"]

# Remap SEX column from numeric to string values
X['sex'] = X['SEX'].map({1: 'male', 2: 'female'})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training baseline models

In [6]:
male_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
male_model.fit(X_train[X_train['sex'] == 'male'], y_train[X_train['sex'] == 'male'])

female_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
female_model.fit(X_train[X_train['sex'] == 'female'], y_train[X_train['sex'] == 'female'])

normal_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
normal_model.fit(X_train, y_train)

print("done")

done


In [7]:
ff_model = CombinedEBM([male_model, female_model], [0.5, 0.5])
ff_model_obj = ff_model.get_model_object()

merged_model = merge_ebms([male_model, female_model])

# Displaying with custom EBMVisualizer

In [8]:
visualizer = InterpretmlEBMVisualizer([male_model, female_model, normal_model, ff_model_obj, merged_model], ["Male Model", "Female Model", "Normal Model", "50-50 Model", "Merged Model"])
visualizer.show()

# Group Performance Plots

In [11]:
male_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
male_model.fit(X_train[X_train['sex'] == 'male'], y_train[X_train['sex'] == 'male'])

female_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
female_model.fit(X_train[X_train['sex'] == 'female'], y_train[X_train['sex'] == 'female'])

normal_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
normal_model.fit(X_train, y_train)

print("done")

done


In [9]:
foi = 'sex'
_x = X_train
_y = y_train

male_mask = _x[foi] == 'male'
female_mask = _x[foi] == 'female'

In [14]:
%matplotlib widget
plt.ioff()
analyzer = GroupPerformanceAnalyzer(
    male_model, female_model, normal_model,
    _x, _y,
    male_mask=male_mask, female_mask=female_mask,
    feature_of_interest='sex',
    combine_strategy='post',
    metric='log_likelihood',
)
analyzer.generate_plot(n_combinations=100)

Evaluating combinations: 100%|██████████| 100/100 [00:04<00:00, 20.68it/s]
